In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

In [3]:
sdi = pd.read_excel('./data/corr/sdi_total.xlsx')

In [4]:
sdi = sdi.dropna(axis = 1)

In [5]:
sdi.head()

,회사명,거래소코드,회계년도,[공통]총자본증가율(IFRS연결),[공통]자기자본증가율(IFRS연결),[공통]매출액증가율(IFRS연결),[공통]정상영업이익증가율(IFRS연결),[공통]순이익증가율(IFRS연결),[공통]총포괄이익증가율(IFRS연결),[공통]매출액정상영업이익률(IFRS연결),[공통]매출액순이익률(IFRS연결),[공통]총자본사업이익률(IFRS연결),[공통]총자본정상영업이익률(IFRS연결),[공통]총자본순이익률(IFRS연결),[공통]자기자본정상영업이익률(IFRS연결),[공통]자기자본순이익률(IFRS연결),[공통]자본금정상영업이익률(IFRS연결),[공통]자본금순이익률(IFRS연결),[공통]영업비용 대 영업수익비율(IFRS연결),[공통]수지비율(관계기업투자손익 제외)(IFRS연결),[공통]세금과공과 대 총비용비율(IFRS연결),[공통]유보율(IFRS연결),[공통]적립금비율(재정비율)(IFRS연결),[공통]1주당매출액(IFRS연결),[공통]1주당순이익(IFRS연결),[공통]1주당 CASH FLOW(IFRS연결),[공통]1주당정상영업이익(IFRS연결),[공통]자기자본구성비율(IFRS연결),[공통]타인자본구성비율(IFRS연결),[공통]부채비율(IFRS연결),[공통]차입금의존도(IFRS연결),[공통]차입금비율(IFRS연결),[공통]유보액대비율(IFRS연결),[공통]CASH FLOW 대 총자본비율(IFRS연결),[공통]총자본회전률(IFRS연결),[공통]자기자본회전률(IFRS연결),[공통]자본금회전률(IFRS연결),[공통]타인자본회전률(IFRS연결),[공통]부가가치(IFRS연결),[공통]총자본투자효율(IFRS연결),[공통]부가가치율(IFRS연결),[공통]노동소득분배율(IFRS연결),[공통]자본분배율(IFRS연결),거래년도(*),거래월(*),거래일수,최고가,최저가,산술평균,종가,거래량,거래대금
0,삼성SDI(주),6400,2009/12,5.60,9.60,-6.62,101.61,459.91,0.00,5.42,4.87,4.46,3.85,3.47,5.42,4.58,111.44,90.43,97.99,95.11,0,2051.43,86.83,104965.03,5060,18122.15,5685.25,71.80,28.20,39.28,13.36,18.61,71.30,11.42,0.71,1.00,20.57,2.47,262998.69,3.68,5.31,0.00,0.00,2009,12,21,151000,125000,141310,148500,11952615,16864431817
1,삼성SDI(주),6400,2010/06,0.00,0.00,0.00,0.00,0.00,0.00,5.86,5.30,4.66,4.03,3.65,5.44,4.37,123.24,95.66,96.93,94.18,0,2167.85,83.13,53660.14,2676,3509.70,3143.62,74.08,25.92,34.99,9.07,12.24,73.31,1.13,0.69,0.93,21.04,2.66,157492.07,2.14,6.22,0.00,0.00,2010,6,21,182500,165500,175571,173000,14286450,24959116260
2,삼성SDI(주),6400,2010/12,10.99,21.69,3.48,6.94,63.61,140.47,5.60,7.52,4.35,3.80,5.11,5.05,6.46,119.17,147.96,96.30,92.55,0,2488.11,76.92,108619.83,8203,10363.38,6079.59,78.54,21.46,27.33,4.52,5.75,77.75,8.47,0.68,0.90,21.29,2.76,408654.95,5.15,7.97,0.00,0.00,2010,12,22,181000,161000,170886,168000,10389944,17795549460
3,삼성SDI(주),6400,2011/06,15.96,17.62,1.07,0.86,43.53,-30.03,5.85,6.97,2.03,1.88,2.24,2.53,2.88,62.15,68.65,102.62,91.84,0,2549.28,76.43,54231.69,3818,-273.50,3170.58,74.68,25.32,33.91,7.44,9.96,73.88,0.89,0.32,0.43,10.63,1.26,770025.40,9.03,30.10,42.81,57.19,2011,6,21,183500,155000,165190,163500,11360780,18986249315
4,삼성SDI(주),6400,2011/12,7.49,1.11,6.24,-28.97,-10.11,-88.36,3.74,6.45,2.48,2.48,4.27,3.25,5.26,84.64,133.00,101.46,92.42,0,2512.93,79.58,115394.57,7341,6353.57,4318.14,74.05,25.95,35.04,8.93,12.06,73.23,4.62,0.66,0.87,22.62,2.78,1589036.41,18.63,29.19,42.31,57.69,2011,12,21,153000,128000,137619,133500,8321983,11515506990


In [14]:
sdi_corr = sdi.corr(method = 'pearson')
df_sdi = pd.DataFrame(sdi_corr['종가']).reset_index().sort_values('종가', ascending = False)
df_sdi.columns = ['지표','종가']
df_sdi

,지표,종가
47,종가,1.000000
46,산술평균,0.987642
44,최고가,0.984903
45,최저가,0.975809
49,거래대금,0.546810
7,[공통]매출액정상영업이익률(IFRS연결),0.535537
14,[공통]자본금정상영업이익률(IFRS연결),0.532343
24,[공통]1주당정상영업이익(IFRS연결),0.520019
12,[공통]자기자본정상영업이익률(IFRS연결),0.505743
10,[공통]총자본정상영업이익률(IFRS연결),0.503506


In [15]:
df_sdi.index

Int64Index([47, 46, 44, 45, 49, 7, 14, 24, 12, 10, 9, 40, 6, 4, 8, 30, 11, 15, 13, 22, 48, 3, 5, 38, 20, 25, 23, 35, 32, 33, 36, 41, 37, 34, 19, 28, 29, 21, 27, 26, 31, 42, 2, 1, 43, 39, 16, 17, 0, 18], dtype='int64')

In [7]:
df_sdi.to_excel('sdi.xlsx')

# 6개월 후 주가

In [44]:
# 3개월 후 주가
# ssg_1['종가(원)'] = ssg_1['종가(원)'][1:].reset_index(drop=True)

In [45]:
ssg_1['종가(원)'] = ssg_1['종가(원)'][2:].reset_index(drop=True)

In [46]:
ssg_1['종가(원)']

0     537000.0
1     614000.0
2     339500.0
3     245000.0
4     210000.0
5     218500.0
6     211000.0
7     252500.0
8     218500.0
9     181000.0
10    268000.0
11    230000.0
12    195000.0
13    176000.0
14    234500.0
15    300000.0
16    401500.0
17         NaN
18         NaN
Name: 종가(원), dtype: float64

In [47]:
ssg_1 = ssg_1.dropna(axis = 0)
ssg_1

,총자본증가율(IFRS연결),자기자본증가율(IFRS연결),매출액증가율(IFRS연결),정상영업이익증가율(IFRS연결),순이익증가율(IFRS연결),총포괄이익증가율(IFRS연결),매출액정상영업이익률(IFRS연결),매출액순이익률(IFRS연결),총자본사업이익률(IFRS연결),총자본정상영업이익률(IFRS연결),총자본순이익률(IFRS연결),자기자본정상영업이익률(IFRS연결),자기자본순이익률(IFRS연결),자본금정상영업이익률(IFRS연결),자본금순이익률(IFRS연결),영업비용 대 영업수익비율(IFRS연결),수지비율(관계기업투자손익 제외)(IFRS연결),세금과공과 대 총비용비율(IFRS연결),유보율(IFRS연결),적립금비율(재정비율)(IFRS연결),1주당매출액(IFRS연결)(원),1주당순이익(IFRS연결)(원),1주당 CASH FLOW(IFRS연결)(원),1주당정상영업이익(IFRS연결)(원),자기자본구성비율(IFRS연결),타인자본구성비율(IFRS연결),부채비율(IFRS연결),차입금의존도(IFRS연결),차입금비율(IFRS연결),유보액대비율(IFRS연결),CASH FLOW 대 총자본비율(IFRS연결),총자본회전률(IFRS연결),자기자본회전률(IFRS연결),자본금회전률(IFRS연결),타인자본회전률(IFRS연결),부가가치(IFRS연결)(백만원),총자본투자효율(IFRS연결),부가가치율(IFRS연결),노동소득분배율(IFRS연결),자본분배율(IFRS연결),거래년도(*),거래월(*),거래일수,최고가(원),최저가(원),산술평균(원),종가(원),거래량(주),거래대금(원)
0,0.00,0.00,0.00,0.00,0.00,0.00,7.93,5.02,9.43,9.22,5.83,23.84,15.07,873.59,530.86,92.07,93.14,0,3421.60,97.28,550477.53,26543,51310.85,43679.67,38.68,61.32,158.52,34.46,89.07,37.63,10.83,1.16,3.00,110.10,1.90,849625.26,9.51,8.18,0.00,0.00,2007,12,18,763000,666000,719444,537000.0,1074834,7725748911
1,20.85,16.02,6.45,9.98,14.07,0.00,8.20,5.09,9.43,9.18,5.71,24.40,15.92,960.76,605.55,91.80,93.60,0,3985.88,98.07,585991.43,30278,31790.50,48037.80,36.77,63.23,171.93,41.06,111.66,35.90,7.26,1.12,2.98,117.20,1.80,909906.91,8.43,8.23,0.00,0.00,2008,12,21,498500,386000,451476,614000.0,1571615,7070606603
2,3.31,15.15,4.00,2.99,-0.82,0.00,8.12,5.07,8.78,8.50,5.31,21.86,13.66,989.53,600.61,91.88,93.32,0,4605.05,97.34,609403.28,30030,51038.02,49476.31,40.96,59.04,144.15,37.22,90.87,40.11,7.00,1.05,2.69,121.88,1.71,965271.72,8.65,8.40,0.00,0.00,2009,12,21,566000,515000,537571,339500.0,1292884,6968542198
3,28.46,64.19,-83.75,-74.39,92.00,0.00,12.80,59.48,2.09,1.88,8.72,3.98,18.55,253.46,1153.16,87.83,81.19,0,7625.18,74.26,99043.79,0,25740.79,12673.04,51.87,48.13,92.78,22.98,44.31,51.21,5.05,0.15,0.31,19.81,0.28,763580.34,5.33,40.88,22.39,77.61,2010,12,22,672000,554000,578682,245000.0,987961,5775921269
4,0.00,0.00,0.00,0.00,0.00,0.00,12.84,312.14,8.25,7.18,174.47,13.62,344.69,574.06,13913.80,88.04,86.46,0,3943.89,423.30,111732.27,120246,-31268.35,14351.61,52.72,47.28,89.66,20.49,38.87,51.56,-3.91,0.56,1.06,44.69,1.18,346313.21,8.80,31.48,25.94,74.06,2011,6,21,407500,270000,318095,210000.0,1503624,5239209057
5,-71.07,-73.03,16.99,11.96,221.99,17.91,12.24,161.16,3.20,2.90,38.13,5.57,75.71,372.88,4879.25,88.89,87.87,0,3898.07,431.77,221984.54,0,-6074.14,27180.56,52.59,47.41,90.15,19.37,36.83,51.49,5.14,0.24,0.45,30.45,0.49,695774.23,16.79,31.84,27.38,72.62,2011,12,21,262000,234000,246024,218500.0,816130,2002264137
6,12.45,6.55,-0.32,-7.37,-97.17,-95.20,11.94,9.84,3.68,3.13,2.58,5.92,4.72,265.88,196.92,89.69,87.27,0,4208.37,72.51,111372.82,9855,-1832.37,13293.99,52.93,47.07,88.92,22.44,42.40,51.90,-3.68,0.26,0.50,22.27,0.56,366473.56,8.28,33.42,27.22,72.78,2012,6,20,235500,205000,218350,211000.0,713648,1559516030
7,68.13,12.35,5.09,-11.62,-95.38,-93.24,10.30,8.44,4.81,4.26,3.49,8.96,7.75,480.43,328.29,89.70,88.74,0,4390.99,73.08,233281.94,16429,25823.04,24021.46,44.50,55.50,124.74,30.51,68.57,43.84,1.40,0.41,0.87,46.66,0.79,764865.55,10.98,33.30,27.42,72.58,2012,12,18,225500,201500,215167,252500.0,439529,944658850
8,66.89,11.53,7.49,13.54,-10.57,1.45,12.61,8.51,2.85,2.52,1.70,5.31,3.87,301.88,176.10,87.39,89.45,0,4704.19,71.57,119711.06,8813,-372.55,15094.19,44.11,55.89,126.69,33.63,76.24,43.50,-0.15,0.20,0.42,23.94,0.38,416986.58,5.64,35.38,27.23,72.77,2013,6,19,235500,206500,218526,218500.0,811477,1771449119
9,7.63,9.20,6.31,29.55,-2.47,-6.35,12.55,7.97,4.61,4.24,2.69,9.51,6.83,622.42,320.19,87.45,89.42,0,4803.58,73.09,248003.25,16024,27825.83,31120.80,44.58,55.42,124.29,31.89,71.52,43.98,3.52,0.34,0.76,49.60,0.61,865821.60,11.54,35.46,27.26,72.74,2013,12,20,258500,239000,248300,181000.0,471697,1167677110


In [ ]:
ssg_corr = nc.corr(method = 'pearson')
df_nc = pd.DataFrame(ssg_corr['종가'].sort_values(ascending = False))
df_nc

In [22]:
# ssg_pct = ssg_1.pct_change()

In [23]:
# ssg_pct_corr = ssg_pct.corr(method = 'pearson')

In [26]:
# ssg_pct_corr['종가(원)'].sort_values(ascending = False)

종가(원)                        1.000000
최고가(원)                       0.970108
산술평균(원)                      0.961772
최저가(원)                       0.910999
거래대금(원)                      0.625513
자본분배율(IFRS연결)                0.500625
거래량(주)                       0.433844
총포괄이익증가율(IFRS연결)             0.382048
자기자본증가율(IFRS연결)              0.372977
유보율(IFRS연결)                  0.363243
유보액대비율(IFRS연결)               0.276536
자기자본구성비율(IFRS연결)             0.267535
1주당순이익(IFRS연결)(원)            0.265287
부가가치율(IFRS연결)                0.241428
총자본증가율(IFRS연결)               0.219269
매출액증가율(IFRS연결)               0.115088
매출액순이익률(IFRS연결)              0.090200
거래년도(*)                      0.075765
매출액정상영업이익률(IFRS연결)           0.073447
부가가치(IFRS연결)(백만원)            0.007244
CASH FLOW 대 총자본비율(IFRS연결)   -0.005939
1주당 CASH FLOW(IFRS연결)(원)    -0.078514
영업비용 대 영업수익비율(IFRS연결)       -0.082139
거래월(*)                      -0.094322
차입금의존도(IFRS연결)              -0.095866
거래일수                        -0.103199
정상영업이익증가율(IF

In [7]:
ssg_corr = ssg.corr(method = 'pearson')

In [9]:
df_ssg = pd.DataFrame(ssg_corr['종가(원)'].sort_values(ascending = False))

In [10]:
df_ssg.to_excel('ssg.xlsx')